In [ ]:
import pylast
import math
import random
import time
from IPython.display import clear_output

API_KEY = "7008b5589d58885ace36d3221e86cbd0"
API_SECRET = "a3f47c877288ee66e42a15bc603cffb6"
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

class Artist:
    #tags -> top three tags
    def __init__(self, ID, name, image, tags = None):
        self.id = ID
        self.name = name
        self.image = image
        self.tags = tags

class Song:

    #image -> cover image
    #tags -> top three tags
    def __init__(self, ID, artist, name, image, tags = None):
        self.id = ID
        self.artist = artist
        self.name = name
        self.image = image
        self.tags = tags

def TryGetCover(track: pylast.Track):
    try:
        return track.get_cover_image()
    except Exception:
        return None
def TryGetImage(artist: pylast.Artist):
    try:
        return artist.get_image()
    except Exception:
        return None

def initialQuery(type):

    query = input(f"Enter search query for favorite {type}: ")

    itemIDs = []
    itemText = []
    itemObjects = []
    count = 0

    if type == "track":
        searchResults = network.search_for_track("", query).get_next_page()
        for item in searchResults[:10]:
            artistName = item.get_artist().get_name()
            trackName = item.get_title()
            itemText.append(f"[{count}]: {trackName} by {artistName}")
            itemIDs.append(f"{artistName} - {trackName}")
            itemObjects.append(item)
            count += 1

    elif type == "artist":
        searchResults = network.search_for_artist(query).get_next_page()
        for item in searchResults[:10]:
            artistName = item.get_name()
            itemText.append(f"[{count}]: {artistName}")
            itemIDs.append(artistName)
            itemObjects.append(item)
            count += 1
    else: 
        return "Invalid query"

    print("-------------RESULTS-------------")
    for entry in itemText:
        print(entry)

    while True:
        try:
            index = int(input(f"Pick a {type} by index (0 to {count - 1}): "))
            if 0 <= index < count:
                break
            print("Invalid index.")
        except ValueError:
            print("Enter a number.")

    selected = itemObjects[index]

    if type == "track":
        track = network.get_track(selected.get_artist().get_name(), selected.get_title())
        title = track.get_title()
        artistId = track.get_artist().get_mbid()
        artistName = track.get_artist().get_name()
        return (track, Song(
                ID = f"{artistName} - {title}",
                artist = artistName,
                name = title,
                image = TryGetCover(track),
                tags = None
            ))
           

    elif type == "artist":
        artist = network.get_artist(selected.get_name())
        id = artist.get_mbid()
        return (artist, Artist(
            ID = id,
            name = artist.get_name(),
            image = TryGetImage(artist), 
            tags = None
        ))






#trackSeed,artistSeed are pylast objects
#I know that just calling get_recommended on like 400 tracks is cheating, so I want to create a track pool with varying sources
#1/4 of the tracks will be generated from lastFM's recommendation algorithm (simple: quarter cheating)
#1/4 of the tracks will be generated from related artists
#get some random artists -> get random albums -> get random songs from each

#outputs custom objects not pylast objects
def generateTrackPool(count: int, trackSeed: pylast.Track, artistSeed: pylast.Artist = None):
    #generate seed integers

    x = 0
    y = 0
    z = 0

    while(True):
        #SYSTEM PARAMETERS
        sigma = count * (1/20)
        mean = count * 1/4
        x = math.floor(random.gauss(mean, sigma))
        y = math.floor(random.gauss(mean, sigma))
        z = count - x - y
        if(x > 0 and y > 0 and z > 0):
            break
            
    pool = set()
    uniqueTags = set() #Filled with Song class from this code
    uniqueArtists = set() #Filled with Artist class from this code
    uniqueArtists.add(Artist(ID = artistSeed.get_mbid() , name = artistSeed.get_name() , image = TryGetImage(artistSeed), tags = None))



    #Step one: generate x similar tracks TO seed track doing BFS-kinda thing
    tempTrack = trackSeed
    done = False
    print(x)
    while not done:
        similarTracks = tempTrack.get_similar(limit = x)
        update = True
        for similar in similarTracks:
            
            track = similar.item #pylist.track object

            if update:
                tempTrack = track
                update = False

            tags = track.get_top_tags()
            firstTags = [l.item.get_name() for l in tags[:3]]
            uniqueTags.update(firstTags)
            pool.add(Song(
                ID = track.get_mbid(),
                artist = track.get_artist().get_name(),
                name = track.get_name(),
                image = TryGetImage(track),
                tags = [l.item.get_name() for l in tags[:3]]
            ))
            if(len(pool) == x - 1):
                done = True
                break

    #throttle for rate limits (IDK if this happens in last fm but it happens in spotify api)
    time.sleep(1)   
    
    #Step two: generate y similar tracks to seed album, BFS kind of thing again
    done = False
    tempArtist = artistSeed
    tempCount = 0
    print(y)

    while not done:
        similarArtists = tempArtist.get_similar(limit=3)
        update = True
        for similar in similarArtists:
            artist = similar.item  #pylast.Artist object

            if update:
                tempArtist = artist
                update = False

            uniqueArtists.add(Artist(
                ID=artist.get_mbid(),
                name=artist.get_name(),
                image=TryGetImage(artist),
                tags=None
            ))
            
            albums = artist.get_top_albums(limit=2)


            for top in albums:
                album = top.item

                #Accounting for edge case of track being a single
                try:
                    tracks = album.get_tracks()
                except pylast.WSError:
                    continue

                randomTracks = [t for t in tracks if random.choice([True, False])] #<- from stack exchange
                for track in randomTracks:
                    tags = track.get_top_tags()
                    firstTags = [l.item.get_name() for l in tags[:3]]
                    uniqueTags.update(firstTags)
                    pool.add(Song(
                        ID=track.get_mbid(),
                        artist=track.get_artist().get_name(),
                        name=track.get_name(),
                        image=TryGetImage(track),
                        tags=firstTags
                    ))
                    tempCount += 1
                    if tempCount == y:
                        done = True
                        break
                if done:
                    break
            if done:
                break



    return pool


(t, tempObjectT) = initialQuery("track")
(a, tempObjectA) = initialQuery("artist")

print(f"Seed track: {tempObjectT.name}")
print(f"Seed artist: {tempObjectA.name}")
clear_output(wait= True)

trackPool = generateTrackPool(40, t, a)
#TRACK POOL HALFWAY DONE BUT USABLE, tracks use song object


print(tempObjectT.name)
for  y in trackPool:
    print(f"{y.name}")

9
11
Fourth of July
Lift Off
Scott Street
Feel the Love
Fire
forwards beckon rebound
Ni**as In Paris
Why I Love You
H•A•M
Drawn to the Blood
New Day
Moon Song
anything
Gotta Have It
The Night We Met
Murder to Excellence
Anchor
No Church in the Wild
Should Have Known Better
That's My Bitch
